# Unsupervised learning-movie recommender

## Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.decomposition import NMF

# Project goal

In this project, you will build a proof of concept: A web application that showcases different movie recommendation algorithms.

1. Download a small version of the MovieLens-dataset


2. Implement a baseline recommender


**3. Derive a user-item matrix**


4. Pick and implement a Collaborative Filtering recommender:

    **a) Collaborative Filtering with Matrix Factorization**
    
    b) Neighbourhood based Collaborative Filtering
    
    
5. Write a flask web interface


6. Connect your recommender-model to flask



# Import data

Import all 4 csv files of the *ml-latest-small.zip* dataset.

In [3]:
ratings = pd.read_csv("ratings.csv")

In [4]:
movies = pd.read_csv("movies.csv")

In [5]:
tags = pd.read_csv("tags.csv")

In [6]:
links = pd.read_csv("links.csv")

Inspect the tables.

In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
ratings.shape

(100836, 4)

In [9]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
movies.shape

(9742, 3)

In [11]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [12]:
tags.shape

(3683, 4)

In [13]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [14]:
links.shape

(9742, 3)

# Merge tables

Merge ratings and movies.

In [15]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [16]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [17]:
movies["movieId"].nunique()

9742

In [18]:
df = pd.merge(movies, ratings, on = "movieId", how = "right")

In [19]:
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,3,Grumpier Old Men (1995),Comedy|Romance,1,4.0,964981247
2,6,Heat (1995),Action|Crime|Thriller,1,4.0,964982224
3,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1,5.0,964983815
4,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1,5.0,964982931


In [20]:
df.shape

(100836, 6)

Drop genres and timestamp columns.

In [21]:
df = df.drop(columns = ["genres", "timestamp"])

In [22]:
df

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,3,Grumpier Old Men (1995),1,4.0
2,6,Heat (1995),1,4.0
3,47,Seven (a.k.a. Se7en) (1995),1,5.0
4,50,"Usual Suspects, The (1995)",1,5.0
...,...,...,...,...
100831,166534,Split (2017),610,4.0
100832,168248,John Wick: Chapter Two (2017),610,5.0
100833,168250,Get Out (2017),610,5.0
100834,168252,Logan (2017),610,5.0


In [23]:
df.isna().sum()

movieId    0
title      0
userId     0
rating     0
dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100836 non-null  int64  
 1   title    100836 non-null  object 
 2   userId   100836 non-null  int64  
 3   rating   100836 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 3.8+ MB


# Filter out movies that have been watched by less than 20 users

In [25]:
df["reviews"] = df.groupby(["title"])["rating"].transform("count")

In [26]:
df.head()

,movieId,title,userId,rating,reviews
0,1,Toy Story (1995),1,4.0,215
1,3,Grumpier Old Men (1995),1,4.0,52
2,6,Heat (1995),1,4.0,102
3,47,Seven (a.k.a. Se7en) (1995),1,5.0,203
4,50,"Usual Suspects, The (1995)",1,5.0,204


In [27]:
df.shape

(100836, 5)

In [28]:
df = df[df["reviews"] > 20][["userId", "title", "movieId", "rating"]]

In [29]:
df.head()

,userId,title,movieId,rating
0,1,Toy Story (1995),1,4.0
1,1,Grumpier Old Men (1995),3,4.0
2,1,Heat (1995),6,4.0
3,1,Seven (a.k.a. Se7en) (1995),47,5.0
4,1,"Usual Suspects, The (1995)",50,5.0


In [30]:
df.shape

(66661, 4)

# Extract film, movieId, rating lists

In [31]:
df = df.sort_values(by = ["movieId"], ascending = True)

In [32]:
df.head()

,userId,title,movieId,rating
0,1,Toy Story (1995),1,4.0
61023,396,Toy Story (1995),1,5.0
22638,155,Toy Story (1995),1,3.0
22684,156,Toy Story (1995),1,4.0
60405,391,Toy Story (1995),1,3.0


In [33]:
df.shape

(66661, 4)

Films.

In [34]:
films = df["title"].unique().tolist()

In [35]:
len(films)

1235

MovieIds.

In [36]:
movieIds = df["movieId"].unique().tolist()

In [37]:
len(movieIds)

1237

Rating.

In [38]:
rates = df.groupby(["movieId"])["rating"].mean().values.tolist()

In [39]:
rates

[3.9209302325581397,
 3.4318181818181817,
 3.2596153846153846,
 3.0714285714285716,
 3.946078431372549,
 3.185185185185185,
 3.496212121212121,
 3.6714285714285713,
 3.926829268292683,
 3.7761194029850746,
 2.727272727272727,
 3.49438202247191,
 3.2222222222222223,
 3.125,
 3.625,
 4.0131578947368425,
 3.1842105263157894,
 3.983050847457627,
 3.65234375,
 3.8358208955223883,
 3.293269230769231,
 2.5434782608695654,
 3.3125,
 3.9753694581280787,
 3.1470588235294117,
 4.237745098039215,
 3.4642857142857144,
 4.027027027027027,
 3.235294117647059,
 3.70625,
 2.532258064516129,
 3.5090909090909093,
 3.0238095238095237,
 3.782608695652174,
 3.4393939393939394,
 3.282608695652174,
 3.326923076923077,
 4.031645569620253,
 4.105769230769231,
 3.5434782608695654,
 2.9516129032258065,
 3.494186046511628,
 3.2450980392156863,
 3.845771144278607,
 3.5454545454545454,
 2.9160583941605838,
 2.806451612903226,
 2.8771929824561404,
 3.6359223300970873,
 3.5606060606060606,
 3.5555555555555554,
 3.0833

# Baseline recommender

In [ ]:
user_query = { 246 : 1, 318 : 3, 50 : 4.5}

In [ ]:
#ratings

In [ ]:
def recommend_popular(query, ratings, k = 10):
    
    """
    
    This function recommends popular movies depeding on their rating.
    
    Parameters:
    -----------
    query: A dictionary referring to ratings of seen movies from one user,
    with movieIds as keys and ratings as values.
    
    ratings: A dataframe with movieIds and their ratings.
    
    k: the number of movies to be recommended.
    """
    user = pd.DataFrame(list(query.items()), columns = ["movieId", "rating"])
    user.set_index("movieId", inplace = True)
    
    ratings = ratings.sort_values(by = "rating", ascending = False)
    ratings_index = ratings.set_index("movieId")
    
    for i in user.index:
        ratings_index.drop(index = [i], inplace = True)
        
    ratings_index.reset_index(inplace = True)
    
    return ratings_index.head(k)

In [ ]:
rec = recommend_popular(user_query, df_ratings, 10)

In [ ]:
rec

In [ ]:
#df

In [ ]:
#df["userId"].max()

In [ ]:
#user

In [ ]:
#user["userId"] = 611

In [ ]:
#seen_movies = user[["movieId"]]
#seen_movies.set_index("movieId", inplace = True)
#seen_movies

In [ ]:
#all_movies = df_ratings[["movieId"]]
#all_movies.set_index("movieId", inplace = True)
#all_movies

In [ ]:
#for i in seen_movies.index:
    #all_movies = all_movies.drop(index = [i])
    #df_test = df_ratings.loc[seen_movies[i]]


# Convert matrix into long format

In [40]:
df_pivot = pd.pivot_table(df, index = "userId", columns = "title", values = "rating" )

In [41]:
df_pivot

title,(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),2001: A Space Odyssey (1968),2012 (2009),...,Young Frankenstein (1974),Young Guns (1988),Zack and Miri Make a Porno (2008),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.5,3.5,NaN


# Impute the missing values

In [42]:
pivot_mean = df_pivot.mean().mean()

In [43]:
pivot_mean

3.5257595098823975

In [44]:
df_pivot = df_pivot.fillna(pivot_mean)

In [45]:
df_pivot.head()

title,(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),2001: A Space Odyssey (1968),2012 (2009),...,Young Frankenstein (1974),Young Guns (1988),Zack and Miri Make a Porno (2008),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,4.00000,3.52576,3.52576,3.52576,...,5.00000,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,4.00000
2,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,...,3.52576,3.52576,3.52576,3.52576,3.00000,3.52576,3.52576,3.52576,3.52576,3.52576
3,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,0.50000,...,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576
4,3.52576,3.52576,3.52576,3.52576,5.00000,3.52576,3.52576,3.52576,3.52576,3.52576,...,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576
5,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,...,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576


In [46]:
df_pivot.shape

(610, 1235)

# NMF

In [47]:
R = df_pivot.values

In [48]:
len(R)

610

Check the number of unique movie genres to decide on number of components for the NMF.

In [49]:
movies["genres"].nunique()

951

In [50]:
m = NMF(n_components = 450, init = "random", random_state = 10, max_iter = 200)

In [51]:
m.fit(R)

/home/nteni/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


NMF(init='random', n_components=450, random_state=10)

In [52]:
Q = m.components_

In [53]:
Q

array([[0.06765953, 0.06549482, 0.04908046, ..., 0.06751632, 0.05110116,
        0.0493535 ],
       [0.00728761, 0.03015367, 0.08589145, ..., 0.06137035, 0.08359071,
        0.0970958 ],
       [0.08634354, 0.01409969, 0.03342461, ..., 0.05197613, 0.00583946,
        0.01350755],
       ...,
       [0.15503988, 0.12051672, 0.1689965 , ..., 0.03741936, 0.09125799,
        0.11590434],
       [0.22422378, 0.17803348, 0.05416064, ..., 0.13072465, 0.23993208,
        0.16108184],
       [0.        , 0.17503296, 0.15501817, ..., 0.31447721, 0.        ,
        0.6260968 ]])

In [54]:
P = m.transform(R)

/home/nteni/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [55]:
P

array([[1.86348975e+00, 8.99395681e-01, 1.20157561e+00, ...,
        9.87619816e-02, 4.90451164e-02, 8.11899895e-03],
       [0.00000000e+00, 4.69710152e-01, 5.94201278e-01, ...,
        3.39036224e-02, 9.25522283e-02, 3.95581995e-03],
       [1.71214523e-01, 4.64903738e-01, 1.92225819e+00, ...,
        1.97154419e-02, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 5.67353318e-01],
       [0.00000000e+00, 1.96244357e-01, 6.13962287e-01, ...,
        1.94519889e-02, 1.41302493e-01, 8.18764386e-03],
       [4.35907577e+00, 3.46550686e+00, 2.27821331e+00, ...,
        2.40179699e-01, 4.25403912e-02, 2.79852472e-03]])

In [56]:
print(m.reconstruction_err_)

40.58906537243212


In [ ]:
#nR = np.dot(P, Q)

In [ ]:
#nR

In [ ]:
#nR.shape

# Predict recommendations for a new user.

## Assign some ratings to a user.

In [133]:
user_query = { "Matrix, The (1999)" : 1, "Shawshank Redemption, The (1994)" : 3, "Jewel of the Nile, The (1985)" : 4.5}

In [134]:
user = pd.DataFrame(list(user_query.items()), columns = ["title", "rating"])

In [135]:
user.set_index("title", inplace = True)

In [136]:
user

,rating
title,
"Matrix, The (1999)",1.0
"Shawshank Redemption, The (1994)",3.0
"Jewel of the Nile, The (1985)",4.5


## Create a film dataframe for all the films in the pivot table.

In [137]:
df_pivot.head()

title,(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),2001: A Space Odyssey (1968),2012 (2009),...,Young Frankenstein (1974),Young Guns (1988),Zack and Miri Make a Porno (2008),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,4.00000,3.52576,3.52576,3.52576,...,5.00000,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,4.00000
2,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,...,3.52576,3.52576,3.52576,3.52576,3.00000,3.52576,3.52576,3.52576,3.52576,3.52576
3,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,0.50000,...,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576
4,3.52576,3.52576,3.52576,3.52576,5.00000,3.52576,3.52576,3.52576,3.52576,3.52576,...,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576
5,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,...,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576,3.52576


In [138]:
mv = df_pivot.columns

In [139]:
mv

Index(['(500) Days of Summer (2009)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1996)',
       '101 Dalmatians (One Hundred and One Dalmatians) (1961)',
       '12 Angry Men (1957)', '13 Going on 30 (2004)',
       '13th Warrior, The (1999)', '1408 (2007)',
       '2001: A Space Odyssey (1968)', '2012 (2009)',
       ...
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Zack and Miri Make a Porno (2008)', 'Zodiac (2007)',
       'Zombieland (2009)', 'Zoolander (2001)', 'Zootopia (2016)',
       'eXistenZ (1999)', 'xXx (2002)', '¡Three Amigos! (1986)'],
      dtype='object', name='title', length=1235)

In [140]:
mv = pd.DataFrame(mv)

In [141]:
mv

,title
0,(500) Days of Summer (2009)
1,10 Things I Hate About You (1999)
2,101 Dalmatians (1996)
3,101 Dalmatians (One Hundred and One Dalmatians...
4,12 Angry Men (1957)
...,...
1230,Zoolander (2001)
1231,Zootopia (2016)
1232,eXistenZ (1999)
1233,xXx (2002)


## Merge user and movies dataframe.

In [144]:
user_merge = pd.merge(mv, user, on = "title", how = "left")

In [145]:
user_merge.shape

(1235, 2)

In [146]:
user_merge.head()

,title,rating
0,(500) Days of Summer (2009),NaN
1,10 Things I Hate About You (1999),NaN
2,101 Dalmatians (1996),NaN
3,101 Dalmatians (One Hundred and One Dalmatians...,NaN
4,12 Angry Men (1957),NaN


### Check if it worked and there are in the dataframe the ratings of the user.

In [147]:
query = user_merge["rating"]

In [148]:
query[:10]

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
Name: rating, dtype: float64

In [149]:
r_count = 0

for i in query:
    if not np.isnan(i):
        r_count += 1
        
r_count

3

In [166]:
user_merge[user_merge["title"] == "Matrix, The (1999)"]

,title,rating
694,"Matrix, The (1999)",1.0


## Predictions

In [152]:
query = query.fillna(0)

In [153]:
query = np.array(query)

In [154]:
len(query)

1235

In [155]:
user2 = np.array(query)

In [156]:
user2 = user2.reshape(1, -1)

In [157]:
user2.shape

(1, 1235)

In [158]:
user2 = m.transform(user2)

In [159]:
user2.shape

(1, 450)

In [160]:
Q.shape

(450, 1235)

In [185]:
rec = np.dot(user2, Q)

In [186]:
rec.shape

(1, 1235)

In [187]:
rec = pd.Series(rec[0], index = df_pivot.columns)

In [188]:
rec

title
(500) Days of Summer (2009)                               0.008402
10 Things I Hate About You (1999)                         0.092740
101 Dalmatians (1996)                                     0.047665
101 Dalmatians (One Hundred and One Dalmatians) (1961)    0.050254
12 Angry Men (1957)                                       0.033546
                                                            ...   
Zoolander (2001)                                          0.036516
Zootopia (2016)                                           0.008998
eXistenZ (1999)                                           0.010378
xXx (2002)                                                0.046999
¡Three Amigos! (1986)                                     0.016191
Length: 1235, dtype: float64

## Exclude the films that the user has seen already.

In [189]:
rec = pd.DataFrame(rec)

In [190]:
rec

,0
title,
(500) Days of Summer (2009),0.008402
10 Things I Hate About You (1999),0.092740
101 Dalmatians (1996),0.047665
101 Dalmatians (One Hundred and One Dalmatians) (1961),0.050254
12 Angry Men (1957),0.033546
...,...
Zoolander (2001),0.036516
Zootopia (2016),0.008998
eXistenZ (1999),0.010378


In [191]:
for i in user.index:
    rec.drop(index = [i], inplace = True)

In [192]:
rec

,0
title,
(500) Days of Summer (2009),0.008402
10 Things I Hate About You (1999),0.092740
101 Dalmatians (1996),0.047665
101 Dalmatians (One Hundred and One Dalmatians) (1961),0.050254
12 Angry Men (1957),0.033546
...,...
Zoolander (2001),0.036516
Zootopia (2016),0.008998
eXistenZ (1999),0.010378


## Recommendation

In [193]:
rec.sort_values(by = 0, ascending = False).head(5)

,0
title,
Monty Python and the Holy Grail (1975),0.204482
Braveheart (1995),0.176821
"Usual Suspects, The (1995)",0.118817
Beauty and the Beast (1991),0.115785
Back to the Future (1985),0.112562
